# ARTIST ANALYSIS: SUCCESS RATES, TIMELINES, METRICS

In [67]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import metrics as mt 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os
load_dotenv()
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_client_id, spotify_client_secret))
df_artists = pd.read_csv("artists_df.csv")
df_timeline = pd.read_csv("timeline_df.csv")
df_artists = df_artists.set_index(df_artists.columns[0]).rename_axis("name", axis = 0)
df_timeline = df_timeline.set_index(df_artists.columns[0]).rename_axis("release_date", axis = 0)
df_artists_ex = df_artists_ex.set_index(df_artists_ex.columns[0]).rename_axis("name")
def show_corr(df, thr = 0.5):
    return (pd.DataFrame(df.corr())[pd.DataFrame(abs(df.corr())) > thr].notnull()*1).sum().sum() - len(pd.DataFrame(df.corr()).index)
def threshold_corr(df, thr = 0.5):
    corr_matrix = pd.DataFrame(df.corr().fillna(0))
    corr_pos = corr_matrix[corr_matrix > thr].notnull()
    corr_neg = corr_matrix[corr_matrix < -thr].notnull()
    columns = indexes = corr_pos.columns
    for row in range(len(indexes)):
        for col in range(len(columns)):
            if(row <= col):
                pass
            elif(corr_pos.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
            elif(corr_neg.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
def find_max(dfx):
    dfr = dfx.transpose()
    for index in dfr.index:
        row = dfr.loc[index]
        print(index, "\n", row[row == row.max()], "\n")
def search_artist(string = "--"):
    results = spotify.search(q='artist:'+string, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        df_artist = pd.Series(artist)
        df_artist = df_artist[df_artist.index.isin(['name', 'followers', 'genres', 'id', 'popularity'])]
        df_artist["followers"] = df_artist["followers"]['total']
        return df_artist

In [68]:
show_corr(df_artists_ex)

10

In [69]:
threshold_corr(df_artists_ex, thr = 0.3)

speechiness  <->  danceability ( 0.4477969837254277 )
speechiness  <->  energy ( 0.39948767525939377 )
acousticness  <->  energy ( -0.6776438385861309 )
acousticness  <->  speechiness ( -0.32810014706144686 )
valence  <->  danceability ( 0.5113944356044473 )
popularity  <->  acousticness ( -0.3361915575605464 )
duration_min  <->  acousticness ( -0.40018664718757296 )
duration_min  <->  valence ( -0.5172650323011795 )
duration_min  <->  popularity ( 0.41240514974805087 )
explicit  <->  danceability ( 0.36808271036539614 )
explicit  <->  speechiness ( 0.5485969981834946 )
explicit  <->  collab ( 0.35810751746996383 )
loudness  <->  energy ( 0.3035409292464316 )
track_count  <->  collab ( -0.7387082081171555 )


In [48]:
threshold_corr(df_artists_ex[["popularity", "acousticness", "duration_min"]], 0.3)

acousticness  <->  popularity ( -0.34680671489572307 )
duration_min  <->  popularity ( 0.41685304330198997 )
duration_min  <->  acousticness ( -0.39990711919146765 )


In [70]:
X = df_artists_ex[["acousticness", "duration_min"]]
y = df_artists_ex["popularity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
print(mt.r2_score(y, linear_model.predict(X)))

0.20124528948494558


In [71]:
df_artists_ex

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,popularity,duration_min,collab,...,genres.1,genres.1.1,genres.2,genres.1.1.1,genres.2.1,genres.1.1.1.1,genres.2.1.1,genres.1.1.1.1.1,genres.2.1.1.1,genres.1.1.1.1.1.1
name,,,,,,,,,,,,,,,,,,,,,
Eminem,0.800684,0.785737,0.220463,0.155246,0.000045,0.272805,0.552600,0.786316,4.578947,0.684211,...,"['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']","['detroit hip hop', 'g funk', 'hip hop', 'rap']"
The Beatles,0.513100,0.495600,0.035740,0.400330,0.025052,0.118130,0.521100,0.770000,3.100000,0.500000,...,"['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe...","['british invasion', 'classic rock', 'merseybe..."
AC/DC,0.518700,0.850500,0.101790,0.057967,0.018540,0.208410,0.515100,0.761000,4.300000,0.500000,...,"['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',...","['album rock', 'australian rock', 'hard rock',..."
Elton John,0.597579,0.593105,0.047653,0.311295,0.013135,0.140721,0.578000,0.750526,4.578947,0.526316,...,"['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so...","['glam rock', 'mellow gold', 'piano rock', 'so..."
Bob Marley & The Wailers,0.796800,0.502000,0.181440,0.171593,0.002398,0.156200,0.708200,0.744000,4.200000,0.500000,...,['reggae'],['reggae'],['reggae'],['reggae'],['reggae'],['reggae'],['reggae'],['reggae'],['reggae'],['reggae']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Shirelles,0.555400,0.540500,0.045190,0.556900,0.000161,0.271470,0.673800,0.416000,2.500000,0.500000,...,"['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla..."
The Miracles,0.529000,0.617444,0.038983,0.571411,0.002932,0.194883,0.645000,0.410556,3.555556,0.555556,...,"['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla...","['adult standards', 'brill building pop', 'cla

In [66]:
df_genres = genre_df(df_artists_ex)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,popularity,duration_min,collab,key,mode,explicit,available_markets,tempo,loudness
genres,,,,,,,,,,,,,,,,
detroit hip hop,0.800684,0.785737,0.220463,0.155246,0.000045,0.272805,0.552600,0.786316,4.578947,0.684211,7,1,1,1.0,7,6
merseybeat,0.513100,0.495600,0.035740,0.400330,0.025052,0.118130,0.521100,0.770000,3.100000,0.500000,9,1,0,1.0,5,5
australian rock,0.518700,0.850500,0.101790,0.057967,0.018540,0.208410,0.515100,0.761000,4.300000,0.500000,9,1,0,1.0,7,6
reggae,0.796800,0.502000,0.181440,0.171593,0.002398,0.156200,0.708200,0.744000,4.200000,0.500000,9,1,0,1.0,6,5
r&b,0.819400,0.725100,0.063480,0.209210,0.054572,0.166990,0.668700,0.740000,4.600000,0.500000,11,1,0,1.0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
british blues,0.423200,0.644000,0.052600,0.190282,0.230894,0.114610,0.550000,0.443000,2.600000,0.500000,0,1,0,1.0,7,5
classic girl group,0.563200,0.592400,0.048810,0.449940,0.000685,0.262760,0.701750,0.429500,2.650000,0.500000,7,1,0,1.0,7,5
alternative country,0.531000,0.434474,0.026963,0.442411,0.000193,0.135779,0.493789,0.417368,4.210526,0.526316,10,1,0,1.0,6,5
